In [0]:
import random
import glob
import subprocess
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow.keras as Ke
from tensorflow.keras.callbacks import Callback
# from tensorflow.keras.layers import BatchNormalization as BN
# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dense, Add, UpSampling2D, Concatenate, Layer, Dropout, MaxPool2D, Cropping2D,Lambda
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import *
import wandb
from wandb.keras import WandbCallback
import cv2
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
val_dir = 'data/test'
train_dir = 'data/train'

# automatically get the data if it doesn't exist
if not os.path.exists("data"):
    print("Downloading flower dataset...")
    subprocess.check_output(
        "mkdir data && curl https://storage.googleapis.com/wandb/flower-enhance.tar.gz | tar xz -C data", shell=True)

In [0]:
import numpy as np
import glob
import cv2
from PIL import Image
import os

dest_path='data/train'

def rotate(image, file_name,rotate_deg):
    rotate_90 = image.rotate(rotate_deg)
#     print(file_name.split('/'))
    rotate_90.save(os.path.join(dest_path,'R9'+file_name.split('/')[2]))

def mirror(image,file_name):
    H_img = image.transpose(Image.FLIP_LEFT_RIGHT)
    V_img = image.transpose(Image.FLIP_TOP_BOTTOM)

    V_img.save(os.path.join(dest_path,'VF'+file_name.split('/')[2]))
    H_img.save(os.path.join(dest_path,'HF'+file_name.split('/')[2]))

def image_auggenerator( img_dir):
    """A generator that returns small images and large images.  DO NOT ALTER the validation set"""
    input_filenames = glob.glob(img_dir + "/*-in.jpg")
    counter = 0
    for i in range (0,len(input_filenames)):
        img=input_filenames[counter]
        in_image=Image.open(img)#cv2.imread(img)
        rotate(in_image,img,90)
        mirror(in_image,img)
        out_image=Image.open(img.replace("-in.jpg", "-out.jpg"))#cv2.imread(img.replace("-in.jpg", "-out.jpg"))
        rotate(out_image,img.replace("-in.jpg", "-out.jpg"),90)
        mirror(out_image,img.replace("-in.jpg", "-out.jpg"))
        counter+=1

img_dir='data/train'

image_auggenerator(img_dir)
print(len(os.listdir('data/train')))

In [0]:
config = run.config
config.num_epochs = 10
config.batch_size = 4
config.input_height = 64
config.input_width = 64
config.output_height = 256
config.output_width = 256


config.steps_per_epoch = len(
    glob.glob(train_dir + "/*-in.jpg")) // config.batch_size
config.val_steps_per_epoch = len(
    glob.glob(val_dir + "/*-in.jpg")) // config.batch_size




def image_generator(batch_size, img_dir):
    """A generator that returns small images and large images.  DO NOT ALTER the validation set"""
    input_filenames = glob.glob(img_dir + "/*-in.jpg")
    counter = 0
    while True:
        small_images = np.zeros(
            (batch_size, config.input_width, config.input_height, 3))
        large_images = np.zeros(
            (batch_size, config.output_width, config.output_height, 3))
        random.shuffle(input_filenames)
        if counter+batch_size >= len(input_filenames):
            counter = 0
        for i in range(batch_size):
            img = input_filenames[counter + i]
            in_img=cv2.imread((img))
            in_img=cv2.resize(in_img,(64,64))
            small_images[i] = np.array(in_img) / 255.0
            large_images[i] = np.array(
                cv2.imread(img.replace("-in.jpg", "-out.jpg"))) / 255.0
        yield (small_images, large_images)
        counter += batch_size


def perceptual_distance(y_true, y_pred):
    """Calculate perceptual distance, DO NOT ALTER"""
    y_true *= 255
    y_pred *= 255
    rmean = (y_true[:, :, :, 0] + y_pred[:, :, :, 0]) / 2
    r = y_true[:, :, :, 0] - y_pred[:, :, :, 0]
    g = y_true[:, :, :, 1] - y_pred[:, :, :, 1]
    b = y_true[:, :, :, 2] - y_pred[:, :, :, 2]

    return K.mean(K.sqrt((((512+rmean)*r*r)/256) + 4*g*g + (((767-rmean)*b*b)/256)))


val_generator = image_generator(config.batch_size, val_dir)
in_sample_images, out_sample_images = next(val_generator)


class ImageLogger(Callback):
    def on_epoch_end(self, epoch, logs):
        preds = self.model.predict(in_sample_images)
        in_resized = []
        for arr in in_sample_images:
            # Simple upsampling
            in_resized.append(arr.repeat(8, axis=0).repeat(8, axis=1))
        wandb.log({
            "examples": [wandb.Image(np.concatenate([in_resized[i] * 255, o * 255, out_sample_images[i] * 255], axis=1)) for i, o in enumerate(preds)]
        }, commit=False)

        

        
        
def ms_ssmi_loss(y_true,y_pred):
  ssim_loss=K.mean(tf.image.ssim_multiscale(y_true,y_pred,1.0))
  l1_loss=K.sum(tf.keras.losses.mean_absolute_error(y_true, y_pred),axis=1)
  loss = 1.5*ssim_loss+l1_loss*(0.5)
  return loss
  


In [0]:
import numpy as np
import glob
import cv2
from PIL import Image
import os

dest_path='data/train'

def rotate(image, file_name,rotate_deg):
    rotate_90 = image.rotate(rotate_deg)
#     print(file_name.split('/'))
    rotate_90.save(os.path.join(dest_path,'R9'+file_name.split('/')[2]))

def mirror(image,file_name):
    H_img = image.transpose(Image.FLIP_LEFT_RIGHT)
    V_img = image.transpose(Image.FLIP_TOP_BOTTOM)

    V_img.save(os.path.join(dest_path,'VF'+file_name.split('/')[2]))
    H_img.save(os.path.join(dest_path,'HF'+file_name.split('/')[2]))

def image_auggenerator( img_dir):
    """A generator that returns small images and large images.  DO NOT ALTER the validation set"""
    input_filenames = glob.glob(img_dir + "/*-in.jpg")
    counter = 0
    for i in range (0,len(input_filenames)):
        img=input_filenames[counter]
        in_image=Image.open(img)#cv2.imread(img)
        rotate(in_image,img,90)
        mirror(in_image,img)
        out_image=Image.open(img.replace("-in.jpg", "-out.jpg"))#cv2.imread(img.replace("-in.jpg", "-out.jpg"))
        rotate(out_image,img.replace("-in.jpg", "-out.jpg"),90)
        mirror(out_image,img.replace("-in.jpg", "-out.jpg"))
        counter+=1

img_dir='data/train'

image_auggenerator(img_dir)
print(len(os.listdir('data/train')))

Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [0]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import h5py

# base_model = VGG19(weights='imagenet')
# print(base_model.summary())
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('relu3_3').output)
#
# img_path = r'C:\Users\vghorpad\OneDrive - Qualcomm\Pictures\medium_exposure.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
#
# block4_pool_features = model.predict(x)
import random
import glob
import subprocess
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow.keras as Ke
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import BatchNormalization as BN
from tensorflow.keras.optimizers import Adam
# import wandb
from tensorflow.keras import *
from tensorflow.keras.layers import *
from wandb.keras import WandbCallback
import cv2
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model,load_model


def icnr_weights(init=tf.glorot_normal_initializer(), scale=2, shape=[3, 3, 32, 4], dtype=tf.float32):
    sess = tf.Session()
    return sess.run(ICNR(init, scale=scale)(shape=shape, dtype=dtype))


class ICNR:
    """ICNR initializer for checkerboard artifact free sub pixel convolution
    Ref:
     [1] Andrew Aitken et al. Checkerboard artifact free sub-pixel convolution
     https://arxiv.org/pdf/1707.02937.pdf)
    Args:
    initializer: initializer used for sub kernels (orthogonal, glorot uniform, etc.)
    scale: scale factor of sub pixel convolution
    """

    def __init__(self, initializer, scale=1):
        self.scale = scale
        self.initializer = initializer

    def __call__(self, shape, dtype, partition_info=None):
        shape = list(shape)
        if self.scale == 1:
            return self.initializer(shape)

        new_shape = shape[:3] + [shape[3] // (self.scale ** 2)]
        x = self.initializer(new_shape, dtype, partition_info)
        x = tf.transpose(x, perm=[2, 0, 1, 3])
        x = tf.image.resize_nearest_neighbor(x, size=(shape[0] * self.scale, shape[1] * self.scale))
        x = tf.space_to_depth(x, block_size=self.scale)
        x = tf.transpose(x, perm=[1, 2, 0, 3])

        return x
data_format='channels_first'

class Subpixel(Conv2D):
    def __init__(self,
                 filters,
                 kernel_size,
                 r,
                 padding='valid',
                 data_format=data_format,
                 strides=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(Subpixel, self).__init__(
            filters=r * r * filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)
        self.r = r

    def _phase_shift(self, I):
        r = self.r
        bsize, a, b, c = I.get_shape().as_list()
        bsize = K.shape(I)[0]  # Handling Dimension(None) type for undefined batch dim
        X = K.reshape(I, [bsize, a, b, int(c / (r * r)), r, r])  # bsize, a, b, c/(r*r), r, r
        X = K.permute_dimensions(X, (0, 1, 2, 5, 4, 3))  # bsize, a, b, r, r, c/(r*r)
        # Keras backend does not support tf.split, so in future versions this could be nicer
        X = [X[:, i, :, :, :, :] for i in range(a)]  # a, [bsize, b, r, r, c/(r*r)
        X = K.concatenate(X, 2)  # bsize, b, a*r, r, c/(r*r)
        X = [X[:, i, :, :, :] for i in range(b)]  # b, [bsize, r, r, c/(r*r)
        X = K.concatenate(X, 2)  # bsize, a*r, b*r, c/(r*r)
        return X

    def call(self, inputs):
        return self._phase_shift(super(Subpixel, self).call(inputs))

    def compute_output_shape(self, input_shape):
        unshifted = super(Subpixel, self).compute_output_shape(input_shape)
        return (unshifted[0], self.r * unshifted[1], self.r * unshifted[2], int(unshifted[3] / (self.r * self.r)))

    def get_config(self):
        config = super(Conv2D, self).get_config()
        config.pop('rank')
        config.pop('dilation_rate')
        config['filters'] = int(config['filters'] / self.r * self.r)
        config['r'] = self.r
        return config

scale=4
def EDSR(scale=4, input_shape=(64, 64, 3), n_feats=64, n_resblocks=16):
    '''
        According to the paper scale can be 2,3 or 4.
        However this code supports scale to be 3 or any of 2^n for n>0
    '''

    def res_block(input_tensor, nf, res_scale=1.0):
        x = Conv2D(nf, (3, 3), padding='same', activation='relu',
                   activity_regularizer=regularizers.l1(10e-10))(input_tensor)
        x = Conv2D(nf, (3, 3), padding='same', activity_regularizer=regularizers.l1(10e-10))(x)
        x = Lambda(lambda x: x * res_scale)(x)
        print(x.shape)
        x = add([x, input_tensor])
        return x

    inp = Input(shape=input_shape)

    x = Conv2D(n_feats, 3, padding='same', activity_regularizer=regularizers.l1(10e-10),data_format=data_format)(inp)
    conv1 = x
    if n_feats == 256:
        res_scale = 0.1
    else:
        res_scale = 1.0
    for i in range(n_resblocks): x = res_block(x, n_feats, res_scale)
    x = Conv2D(n_feats, 3, padding='same', activity_regularizer=regularizers.l1(10e-10),data_format=data_format)(x)
    x = add([x, conv1])

    if not scale % 2:
        for i in range(int(np.log2(scale))):
            x = Subpixel(n_feats, 3, 2, padding='same', activity_regularizer=regularizers.l1(10e-10),data_format=data_format)(x)
    else:  # scale = 3
        x = Subpixel(n_feats, 3, 3, padding='same', activity_regularizer=regularizers.l1(10e-10),data_format=data_format)(x)
    sr = Conv2D(input_shape[-1], 1, padding='same',
                activity_regularizer=regularizers.l1(10e-10),data_format=data_format)(x)

    model = Model(inputs=inp, outputs=sr, name='SR')
    """ICNR initializer for checkerboard artifact free sub pixel convolution
        Ref:
         [1] Andrew Aitken et al. Checkerboard artifact free sub-pixel convolution
         https://arxiv.org/pdf/1707.02937.pdf)
    """
    for layer in model.layers:
        if type(layer) == Subpixel:
            c, b = layer.get_weights()
            if scale == 3:
                w = icnr_weights(scale=3, shape=c.shape, dtype = tf.float32)
            else:
                w = icnr_weights(scale=2, shape=c.shape, dtype = tf.float32)
            layer.set_weights([w, b])
    return model


model = EDSR()
model.summary()
model.load_weights('edsr-16-x4-gen-pre-psnr-28.8885.h5')

In [0]:
model.compile(optimizer=Adam(lr=0.005), loss='mae', metrics=[perceptual_distance])

model.fit_generator(image_generator(config.batch_size, train_dir),
                    steps_per_epoch=config.steps_per_epoch,
                    epochs=config.num_epochs, callbacks=[
                        ImageLogger(), WandbCallback()],
                    validation_steps=config.val_steps_per_epoch,
                    validation_data=val_generator)